In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np

In [17]:
IMG_SIZE = (128, 128)  # Reduced size to save memory
BATCH_SIZE = 16         # Smaller batch size
SEED = 42
DATA_DIR = "/content/drive/MyDrive/potato_disease/training/potato_disease_dataset"

In [18]:
full_ds = image_dataset_from_directory(
    DATA_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True
)

Found 2192 files belonging to 3 classes.


In [19]:
class_names = full_ds.class_names
n_classes = len(class_names)

In [20]:
cardinality = tf.data.experimental.cardinality(full_ds).numpy()
train_size = int(0.7 * cardinality)
val_size = int(0.15 * cardinality)

train_ds = full_ds.take(train_size)
rest = full_ds.skip(train_size)
val_ds = rest.take(val_size)
test_ds = rest.skip(val_size)

In [21]:
normalizer = layers.Rescaling(1./255)
def preprocess(ds):
    return ds.map(lambda x, y: (normalizer(x), y)).prefetch(tf.data.AUTOTUNE)

train_ds = preprocess(train_ds)
val_ds = preprocess(val_ds)
test_ds = preprocess(test_ds)

In [22]:
input_shape = IMG_SIZE + (3,)

encoder_input = layers.Input(shape=input_shape)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(128, 3, activation='relu', padding='same')(encoded)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
decoded = layers.Conv2D(3, 3, activation='sigmoid', padding='same')(x)

autoencoder = Model(encoder_input, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [23]:
ae_train = train_ds.map(lambda x, y: (x, x))
ae_val = val_ds.map(lambda x, y: (x, x))

early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_ae = autoencoder.fit(
    ae_train,
    validation_data=ae_val,
    epochs=3,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/3
95/95 ━━━━━━━━━━━━━━━━━━━━ 16s 116ms/step - loss: 0.0237 - val_loss: 0.0071
Epoch 2/3
95/95 ━━━━━━━━━━━━━━━━━━━━ 17s 104ms/step - loss: 0.0067 - val_loss: 0.0052
Epoch 3/3
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 101ms/step - loss: 0.0051 - val_loss: 0.0046


In [24]:
encoder = Model(inputs=encoder_input, outputs=encoded)

In [25]:
from tensorflow.keras.layers import GlobalAveragePooling2D

def encode_and_flatten(dataset):
    features = []
    labels = []
    for x_batch, y_batch in dataset:
        encoded = encoder.predict(x_batch)
        pooled = GlobalAveragePooling2D()(encoded).numpy()
        features.append(pooled)
        labels.append(y_batch.numpy())
    return np.vstack(features), np.concatenate(labels)

X_train, y_train = encode_and_flatten(train_ds)
X_val, y_val = encode_and_flatten(val_ds)
X_test, y_test = encode_and_flatten(test_ds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━

In [36]:
classifier = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(n_classes, activation='softmax')
])

classifier.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

history_clf = classifier.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=75,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/75
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4743 - loss: 1.0042 - val_accuracy: 0.5031 - val_loss: 0.8554
Epoch 2/75
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5439 - loss: 0.8271 - val_accuracy: 0.7500 - val_loss: 0.7956
Epoch 3/75
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7577 - loss: 0.7439 - val_accuracy: 0.7875 - val_loss: 0.7274
Epoch 4/75
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7844 - loss: 0.7186 - val_accuracy: 0.8000 - val_loss: 0.6644
Epoch 5/75
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8121 - loss: 0.6287 - val_accuracy: 0.8125 - val_loss: 0.6156
Epoch 6/75
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8126 - loss: 0.6030 - val_accuracy: 0.8062 - val_loss: 0.5736
Epoch 7/75
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8129 - loss: 0.5688 - val_accuracy: 0.8094 - val_loss: 0.5439
Epoch 8/75
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8190 - loss: 0.5369 - val_accuracy: 0.8219 - val_loss

In [39]:
test_loss, test_acc = classifier.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9079 - loss: 0.2565 
Test Accuracy: 0.8977, Loss: 0.2833
